In [37]:
from nicess.bifrost import Tank, Channel, BIFROST
import scipp as sc
import numpy as np
from datetime import date

In [19]:
def explore(obj):
    print(f"{type(obj)}:")
    if type(obj) is None:
        return
    d = np.array([x for x in dir(obj) if x[0] != '_'])
    if len(d) == 0:
        return
    t = len(d)//3
    mw = max(len(x) for x in d)
    d = ["{x:{n}s}".format(x=x,n=mw) for x in d]
    ds = d[:t], d[t+1:2*t], d[2*t+1:]
    print('\n'.join(['\t'.join(l) for l in zip(*ds)]))

def spelunk(obj, prepend=0, max_depth=6):
    if prepend > max_depth:
        return
    if type(obj) is None:
        return
    attrs = [x for x in dir(obj) if x[0] != '_']
    if len(attrs) == 0:
        return
    mw = max(len(x) for x in attrs)
    for attr in attrs:
        child = getattr(obj, attr)
        print("{p}{x:{n}s} = {t}".format(p='.'*prepend, x=attr, n=mw, t=type(child)))
        if not isinstance(child, (str, list, tuple, dict)):
            spelunk(child, prepend+2, max_depth)

In [3]:
obj0 = Channel.from_calibration(sc.scalar(0., unit='degree'))
obj0_cq = obj0.to_cadquery()

In [4]:
obj = Tank.from_calibration()
obj_cq = obj.to_cadquery()
#obj_cq.save('test_BIFROST_secondary.step')

In [5]:
# from jupyter_cadquery.viewer.client import show, show_object

In [6]:
# show(obj0_cq)

In [7]:
# obj0_cq

In [38]:
obj_cq.save(f'{date.isoformat(date.today())}_BIFROST_secondary.step')


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : 2022-10-05_BIFROST_secondary.step(135480 ents)  Write  Done


In [20]:
explore(obj_cq)

<class 'cadquery.assembly.Assembly'>:
add        	loc        	parent     
children   	metadata   	save       
color      	name       	shapes     
constrain  	obj        	solve      


In [10]:
explore(obj_cq.children[0])

<class 'cadquery.assembly.Assembly'>:
add        	loc        	parent     
children   	metadata   	save       
color      	name       	shapes     
constrain  	obj        	solve      


In [11]:
explore(obj_cq.children[0].obj)

<class 'cadquery.occ_impl.shapes.Compound'>:
Area                    	copy                    	isValid                 
BoundingBox             	cut                     	label                   
Center                  	distance                	locate                  
CenterOfBoundBox        	distances               	located                 
Closed                  	dprism                  	location                
CombinedCenter          	exportBrep              	makeCompound            
CombinedCenterOfBoundBox	exportStep              	makeText                
CompSolids              	exportStl               	mesh                    
Compounds               	facesIntersectedByLine  	mirror                  
Edges                   	fillet                  	move                    
Faces                   	fix                     	moved                   
ShapeType               	forConstruction         	remove                  
Shells                  	fuse                    	rotat

In [13]:
obj_cq.objects

{'faacc10a-4490-11ed-81f3-acde48001122': <cadquery.assembly.Assembly at 0x1b64012e0>,
 'channel-0': <cadquery.assembly.Assembly at 0x1b80546a0>,
 'channel-1': <cadquery.assembly.Assembly at 0x1b80539d0>,
 'channel-2': <cadquery.assembly.Assembly at 0x1b804d7f0>,
 'channel-3': <cadquery.assembly.Assembly at 0x1b6403b80>,
 'channel-4': <cadquery.assembly.Assembly at 0x1b80532b0>,
 'channel-5': <cadquery.assembly.Assembly at 0x1b8040670>,
 'channel-6': <cadquery.assembly.Assembly at 0x1b8dbf460>,
 'channel-7': <cadquery.assembly.Assembly at 0x1b8dc24c0>,
 'channel-8': <cadquery.assembly.Assembly at 0x1b8058430>}

In [34]:
def com_xy_angle(obj):
    from numpy import arctan2, pi
    v = obj.centerOfMass(obj)
    return arctan2(v.y, v.x) / pi * 180

In [40]:
print(' '.join([f"{y: 3.2f}" for y in map(lambda x: com_xy_angle(x.obj), obj_cq.children)]))

-39.96 -29.98 -19.99 -10.00  0.00  10.01  20.02  30.04  40.06


In [41]:
ch4 = obj_cq.objects['channel-4']

In [42]:
explore(ch4.obj)

<class 'cadquery.occ_impl.shapes.Compound'>:
Area                    	copy                    	isValid                 
BoundingBox             	cut                     	label                   
Center                  	distance                	locate                  
CenterOfBoundBox        	distances               	located                 
Closed                  	dprism                  	location                
CombinedCenter          	exportBrep              	makeCompound            
CombinedCenterOfBoundBox	exportStep              	makeText                
CompSolids              	exportStl               	mesh                    
Compounds               	facesIntersectedByLine  	mirror                  
Edges                   	fillet                  	move                    
Faces                   	fix                     	moved                   
ShapeType               	forConstruction         	remove                  
Shells                  	fuse                    	rotat

In [46]:
ch4s0 = ch4.obj.Solids()[0]

In [47]:
explore(ch4s0)

<class 'cadquery.occ_impl.shapes.Solid'>:
Area                     	dprism                   	location                 
BoundingBox              	exportBrep               	makeBox                  
Center                   	exportStep               	makeCone                 
CenterOfBoundBox         	exportStl                	makeCylinder             
Closed                   	extrudeLinear            	makeLoft                 
CombinedCenter           	extrudeLinearWithRotation	makeSolid                
CombinedCenterOfBoundBox 	facesIntersectedByLine   	makeSphere               
CompSolids               	fillet                   	makeTorus                
Compounds                	fix                      	makeWedge                
Edges                    	forConstruction          	mesh                     
Faces                    	fuse                     	mirror                   
ShapeType                	geomType                 	move                     
Shells                

In [57]:
ch4s0.Shells()[0].Faces()[0].Edges()[0].Vertices()

In [60]:
ch4s0.facesIntersectedByLine?

Signature:
ch4s0.facesIntersectedByLine(
    point: Union[ForwardRef('Vector'), Tuple[Union[int, float], Union[int, float]], Tuple[Union[int, float], Union[int, float], Union[int, float]]],
    axis: Union[ForwardRef('Vector'), Tuple[Union[int, float], Union[int, float]], Tuple[Union[int, float], Union[int, float], Union[int, float]]],
    tol: float = 0.0001,
    direction: Optional[Literal['AlongAxis', 'Opposite']] = None,
)
Docstring:
Computes the intersections between the provided line and the faces of this Shape

:point: Base point for defining a line
:axis: Axis on which the line rest
:tol: Intersection tolerance
:direction: Valid values : "AlongAxis", "Opposite", if specified will ignore all faces that are not in the specified direction
including the face where the :point: lies if it is the case

:returns: A list of intersected faces sorted by distance from :point:
File:      /usr/local/Caskroom/miniconda/base/envs/calib-cq/lib/python3.9/site-packages/cadquery/occ_impl/shapes.py